In [70]:
# make csv ready to datasets fromat
import pandas as pd

d_a = pd.read_csv("data/twitter_data/labels_tweets_dropna.csv")

from sklearn.model_selection import train_test_split
d_a['label']=(d_a.iloc[:,1:8].sum(axis=1) > 0).astype(int)

d_train, d_test = train_test_split(d_a_r, test_size=0.8, random_state=42)
d_test1, d_val = train_test_split(d_test, test_size=0.5, random_state=42)
d_train.shape, d_test1.shape, d_val.shape
d_train.iloc[:,1:].to_csv("data/twitter_data/tweet_train.csv",index=False)
d_test1.iloc[:,1:].to_csv("data/twitter_data/tweet_test.csv",index=False)
d_val.iloc[:,1:].to_csv("data/twitter_data/tweet_val.csv",index=False)

In [80]:
from datasets import load_dataset

data_files = {"train": "data/twitter_data/tweet_train.csv", "test": "data/twitter_data/tweet_test.csv", "val": "data/twitter_data/tweet_val.csv"}
dataset = load_dataset("csv", data_files=data_files)


Using custom data configuration default-15a9cf023f4f5ab3
Found cached dataset csv (C:/Users/bruger/.cache/huggingface/datasets/csv/default-15a9cf023f4f5ab3/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
100%|██████████| 3/3 [00:00<00:00, 323.54it/s]


In [81]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

loading configuration file config.json from cache at C:\Users\bruger/.cache\huggingface\hub\models--bert-base-cased\snapshots\a8d257ba9925ef39f3036bfc338acf5283c512d9\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file vocab.txt from cache at C:\Users\bruger/.cache\huggingface\hub\models--bert-base-cased\snapshots\a8d257ba9925ef39f3036bfc338acf5283c512d9\vocab.txt
l

In [99]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

Loading cached shuffled indices for dataset at C:/Users/bruger/.cache/huggingface/datasets/csv/default-15a9cf023f4f5ab3/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-d0c4c758af366569.arrow
Loading cached shuffled indices for dataset at C:/Users/bruger/.cache/huggingface/datasets/csv/default-15a9cf023f4f5ab3/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-c2d45e0058f3a5cb.arrow


In [100]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

loading configuration file config.json from cache at C:\Users\bruger/.cache\huggingface\hub\models--bert-base-cased\snapshots\a8d257ba9925ef39f3036bfc338acf5283c512d9\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file pytorch_model.bin from cache at C:\Users\bruger/.cache\huggingface\hub\models--bert-base-cased\snapshots\a8d257ba9925ef39f3036bfc338acf5283c5

MemoryError: 

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [101]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [102]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [103]:
trainer.train()

***** Running training *****
  Num examples = 100
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 39
  Number of trainable parameters = 108311810

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Tweet. If Tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.


RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:76] data. DefaultCPUAllocator: not enough memory: you tried to allocate 12582912 bytes.